# 2.1.5.	Synthetic - Edge Detection and Depth to Source#

Additional useful information to aid geologic interpretation can be derived from a suite of high quality magnetic data, again using some simple techniques. Source edges and depths can be estimated from magnetic gradients where there is sufficient magnetic susceptibility contrast between adjacent geologic bodies or features. 

Analyzing the depth to a magnetic source is valuable for many reasons. Having an approximation of the depth of a body, especially if it is expected to represent ore, is invariably useful for planning drill targets. For some geologic bodies, such as porphyry intrusions, the shape or topography of the intrusion can indicate were the regions of highest fluid flow might have occurred. Results of depth to source modelling can be used as a guide and starting point for 3D geological modelling or further exploration of the shape of the anomaly using forward modelling or inversion techniques. Depending of course on the physical properties, specifically the magnetic susceptibility, of the rock types within a study area, and their distribution and orientation, it is possible to find the depths to not only compact magnetic bodies, but depth to basement below cover, or depths to magnetic stratigraphic horizons.

For this exercise, we will find the edges and depths of both the small high magnetic susceptibility body and the larger magnetic block in the 3D synthetic geologic model. We will use the **tilt-depth method** (Salem et al., 2007, Blakely et al., 2016) to do this. The tilt angle, discussed in Notebook 2.1.3., considers the vertical and horizontal magnetic gradients. Depth can be estimated where the tilt angle is zero, at the edge or margin of a feature or source. Depth to the edge of a source is then approximately half the distance between the +45 degree and -45 degree contours of the calculated tilt angle.

Inclination must be 90, thus reduced to pole data is used in this calculation. This method assumes that the magnetic body is simple and also vertically-sided, which means that the depth estimation will be less accurate for dipping bodies or bodies with complex margins. 



## Total magnetic intensity data and upward continuation

The below image shows the total magnetic intensity (TMI) that was calculated from the synthetic geologic model. You can use the 'UpDist' slider bar to change the upward continuation distance if it is of interest to identify edges and depths of deeper, larger, or more regional-scale bodies. 

In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Simulator, DataIO
dataFile = DataIO.fetchData(file="./assets/Synthetic/DataGrid.tiff", dtype='GeoTiff', loadFile=True)
display(dataFile)

./assets/Synthetic/DataGrid.tiff


interactive(children=(RadioButtons(description='File Type:', options=('Local', 'Cloud'), value='Local'), Text(value='./assets/Search/', description='Path:'), ToggleButton(value=False, description='Download', icon='check', tooltip='Description'), ToggleButton(value=False, description='Check folder', icon='check', tooltip='Description'), Dropdown(description='Files:', options=('./assets/Synthetic/DataGrid.tiff',), value='./assets/Synthetic/DataGrid.tiff'), RadioButtons(description='File Type:', index=1, options=('CSV', 'GeoTiff', 'GRD'), value='GeoTiff'), ToggleButton(value=True, description='Load File', icon='check', tooltip='Description'), Output()), _dom_classes=('widget-interact',))

In [2]:
dataGrid = Simulator.dataGridGeoref(dataFile.result[0], inc=90, dec=0)
display(dataGrid)

interactive(children=(Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='RdBu_r'), FloatText(value=26909.0, description='EPSG code:'), FloatText(value=90.0, description='Inclination angle positive downward from horizontal:'), FloatText(value=0.0, description='Declination angle positve clockwise from North:'), ToggleButton(value=False, description='Fetch Inc/Dec', icon='check', tooltip='Connect to NOAA API'), ToggleButton(value=False, description='Reduce to pole', icon='check', tooltip='Transform to RTP data'), Output()), _dom_classes=('widget-interact',))

In [3]:
view = Simulator.gridTilt2Depth(dataGrid.result, HSTransp=0, ColorMap='Greys_r', shapeFile='./assets/Synthetic/Synthetic_Zcontours.shp')
display(view)

HBox(children=(VBox(children=(FloatSlider(value=90.0, continuous_update=False, description='SunAzimuth', max=360.0, step=5.0), FloatSlider(value=15.0, continuous_update=False, description='SunAngle', max=90.0, step=5.0), FloatSlider(value=0.9, continuous_update=False, description='ColorTransp', max=1.0, step=0.05), FloatSlider(value=0.0, continuous_update=False, description='HSTransp', max=1.0, step=0.05), FloatSlider(value=5.0, continuous_update=False, description='vScale', max=10.0, min=1.0, step=1.0), Dropdown(description='ColorMap', index=4, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='Greys_r'), Dropdown(description='Grid Filters', index=1, options=('TMI', 'tiltAngle'), value='tiltAngle'), FloatSlider(value=0.0, continuous_update=False, description='UpC Height', max=200.0, step=10.0), Text(value='./Output/MyGeoTiff', description='GeoTiff name:'), ToggleButton(value=False, description='Export GeoTiff', icon='check', tooltip='Description'), Dropdown(description='Depth Color', options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='viridis'), FloatSlider(value=1.0, continuous_update=False, description='Marker Size', max=10.0, min=0.1), Text(value='./Output/EstimatedDepth', description='Shapefile name:'), ToggleButton(value=False, description='Export Shapefile', icon='check', tooltip='Description')), layout=Layout(margin='0px 0px 0px 0px', width='35%')), VBox(children=(Output(),), layout=Layout(height='800px', margin='0px 0px 0px 0px', width='65%'))))